In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import os
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn import metrics
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

# Load data

In [2]:
fifa = pd.read_csv("FIFA19data.csv", sep=r'\s*,\s*', engine='python')
fifa.head()

,ID,Name,Age,Nationality,Overall,Potential,Club,Value,Wage,International Reputation,...,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes
0,158023,L. Messi,31,Argentina,94,94,FC Barcelona,�110.5M,�565K,5.0,...,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0
1,20801,Cristiano Ronaldo,33,Portugal,94,94,Juventus,�77M,�405K,5.0,...,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0
2,190871,Neymar Jr,26,Brazil,92,93,Paris Saint-Germain,�118.5M,�290K,5.0,...,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0
3,193080,De Gea,27,Spain,91,93,Manchester United,�72M,�260K,4.0,...,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0
4,192985,K. De Bruyne,27,Belgium,91,92,Manchester City,�102M,�355K,4.0,...,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0


In [3]:
fifa.columns = fifa.columns.str.replace(" +", "_", regex = True)

# Clean the data

In [4]:
# remove uninterested variables
fifa = fifa.drop('ID', 1)
fifa = fifa.drop('Name', 1)
fifa = fifa.drop('Nationality', 1)
fifa = fifa.drop('Club', 1)
fifa = fifa.drop('Value', 1)
fifa = fifa.drop('Wage', 1)
fifa = fifa.drop('Body_Type', 1)
fifa = fifa.drop('Potential', 1)

In [5]:
for col in fifa.columns:
    fifa[col].fillna(value=fifa[col].mode()[0], inplace=True)  # impute missing values using mode

In [6]:
for col in fifa.columns[0:9]:
    print("This is column ", col)
    print("unique values:", fifa[col].unique())

This is column  Age
unique values: [31 33 26 27 32 25 29 28 24 30 19 40 22 23 34 35 36 37 21 18 20 39 41 17
 38 45 42 16 44]
This is column  Overall
unique values: [94 92 91 90 89 88 87 86 85 84 83 82 81 80 79 78 77 76 75 74 73 72 71 70
 69 68 67 66 65 64 63 62 61 60 59 58 57 56 55 54 53 52 51 50 49 48 47 46]
This is column  International_Reputation
unique values: [5. 4. 3. 2. 1.]
This is column  Weak_Foot
unique values: [4. 5. 3. 2. 1.]
This is column  Skill_Moves
unique values: [4. 5. 1. 3. 2.]
This is column  Work_Rate
unique values: ['Medium/ Medium' 'High/ Low' 'High/ Medium' 'High/ High' 'Medium/ High'
 'Medium/ Low' 'Low/ High' 'Low/ Medium' 'Low/ Low']
This is column  Position
unique values: ['RF' 'ST' 'LW' 'GK' 'RCM' 'LF' 'RS' 'RCB' 'LCM' 'CB' 'LDM' 'CAM' 'CDM'
 'LS' 'LCB' 'RM' 'LAM' 'LM' 'LB' 'RDM' 'RW' 'CM' 'RB' 'RAM' 'CF' 'RWB'
 'LWB']
This is column  Contract_Valid_Until
unique values: ['2021' '2022' '2020' '2023' '2019' '2024' '30-Jun-19' '2025' '2026'
 '31-Dec-18' '2018'

In [7]:
fifa["Work_Rate_good"] = fifa["Work_Rate"].str.split("/", expand = True)[0].str.strip()
fifa["Work_Rate_bad"] = fifa["Work_Rate"].str.split("/", expand = True)[1].str.strip()
fifa = fifa.drop('Work_Rate', 1)

In [8]:
fifa["Contract_Valid_Until"] = fifa["Contract_Valid_Until"].str.replace(".*-", "20", regex = True)

In [9]:
factors = ['International_Reputation', 'Weak_Foot', 'Skill_Moves', 'Position', 'Contract_Valid_Until', 'Work_Rate_good', 'Work_Rate_bad']
for var in factors:
    cat_list = pd.get_dummies(fifa[var], prefix=var)
    fifa = pd.concat([fifa,cat_list], axis = 1)
    fifa = fifa.drop(var, 1)
    fifa = fifa.iloc[:,0:len(fifa.columns)-1]

In [10]:
X = fifa.copy()
X = X.drop('Overall', 1)
Y = fifa.copy()
Y = Y['Overall']

In [11]:
X.head(3)

,Age,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,...,Contract_Valid_Until_2020,Contract_Valid_Until_2021,Contract_Valid_Until_2022,Contract_Valid_Until_2023,Contract_Valid_Until_2024,Contract_Valid_Until_2025,Work_Rate_good_High,Work_Rate_good_Low,Work_Rate_bad_High,Work_Rate_bad_Low
0,31,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,...,0,1,0,0,0,0,0,0,0,0
1,33,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,...,0,0,1,0,0,0,1,0,0,1
2,26,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,...,0,0,1,0,0,0,1,0,0,0


In [12]:
Y.head(3)

0    94
1    94
2    92
Name: Overall, dtype: int64

# Q1
#### Randomly split the data into test and training sets such that only 10% of the records are in the training set.
#### Fit a simple linear regression model to predict the overall score of a player and test your model against the test set. 
#### Calculate the R^2 for the predictions you made on the test set. How many features are used in this model?

In [13]:
# 10% in training set and 90% in test set
X_train,X_test,y_train,y_test=train_test_split(X,Y, test_size=0.9, random_state=31)

Simple linear regression model

In [14]:
lm1 = LinearRegression()
lm1.fit(X_train, y_train)
lm1_predictions = lm1.predict(X_test)              # Use 10% train data to train lm1 and predict 90% test data 
lm1_r2 = r2_score(y_test,lm1_predictions)
print("The R^2 for the predictions on the test set is: ", lm1_r2)

The R^2 for the predictions on the test set is:  0.8903296486814734


In [15]:
coeff_used1 = np.sum(lm1.coef_!=0)
print("The number of features in model lm1 is: ", coeff_used1)

The number of features in model lm1 is:  85


# Q2
#### Fit a simple linear regression model with 5-fold cross validation and predict the overall score of players in the test set. 
#### Calculate R^2 for the predictions and compare with the R^2 from Q1. 

In [16]:
lmcv = cross_validate(lm1, X_test, y_test, cv=5, scoring=('r2', 'neg_mean_squared_error'),
                      return_train_score=True)
lmcv

{'fit_time': array([0.1060648 , 0.07323623, 0.02944803, 0.03090096, 0.0310421 ]),
 'score_time': array([0.00512099, 0.00271177, 0.00257611, 0.00234294, 0.00252485]),
 'test_r2': array([0.89619912, 0.89501323, 0.89464921, 0.89826485, 0.88980994]),
 'train_r2': array([0.89593156, 0.8961087 , 0.89624749, 0.89534702, 0.89742982]),
 'test_neg_mean_squared_error': array([-4.84827414, -5.12426786, -4.8772421 , -4.98889166, -5.19249438]),
 'train_neg_mean_squared_error': array([-4.97647604, -4.91332338, -4.97203373, -4.94338024, -4.89397116])}

In [17]:
lmcv_r2 = lmcv['test_r2'].mean()
lmcv_mse = -lmcv['test_neg_mean_squared_error'].mean()

In [18]:
lm1_mse = mean_squared_error(y_test, lm1_predictions)

In [19]:
lm1rfe = RFE(lm1, 60)
lm1rfecv = cross_validate(lm1rfe, X_test, y_test, cv = 5,
                         scoring=('r2', 'neg_mean_squared_error'), return_train_score=True)
lm1rfecv_r2 = lm1rfecv['test_r2'].mean()
lm1rfecv_mse = -lm1rfecv['test_neg_mean_squared_error'].mean()

In [36]:
pd.DataFrame([[lm1_r2, lmcv_r2, lm1rfecv_r2],
             [lm1_mse, lmcv_mse, lm1rfecv_mse],
             [coeff_used1, coeff_used1, 60]],
             columns = ["model 1 - linear regression",
                        "model 2.1 - linear regression with CV",
                        "model 2.3 - linear regression with RFE and CV"], 
             index = ["R^2 on test set", "MSE on test set", "number of features used"])
# R^2 increases after applying cross validation 

,model 1 - linear regression,model 2.1 - linear regression with CV,model 2.3 - linear regression with RFE and CV
R^2 on test set,0.890330,0.894787,0.889298
MSE on test set,5.219964,5.006234,5.266809
number of features used,85.000000,85.000000,60.000000


# Q3
#### Using the training data from question 1, fit a Lasso regression (use default alpha) to predict the overall scores of players in the test set.
#### How many features are being used by the model? 
#### Calculate the R^2 for the predictions you made on the test set and compare with the R^2 from Q1. 

In [21]:
lasso = Lasso()
lasso.fit(X_train,y_train)
lasso1_predictions = lasso.predict(X_test)
lasso_train_score=lasso.score(X_train,y_train)
lasso_test_score=lasso.score(X_test,y_test)
coeff_used3 = np.sum(lasso.coef_!=0)

In [22]:
print("lasso number of features used: ", coeff_used3)
print("lasso training score:", lasso_train_score)
print("lasso test score: ", lasso_test_score)

lasso number of features used:  23
lasso training score: 0.8584100702105248
lasso test score:  0.8508221432029555


In [23]:
# checking adjusted R2
lm1_train_score = r2_score(y_train, lm1.predict(X_train))   # .score() returns the coefficient of determination R^2 of the prediction.
lm1_train_r2adj = 1-(1-lm1_train_score)*((len(X_train)-1)/(len(X_train)-coeff_used1-1))   # adjusted R^2

lasso_train_r2adj = 1-(1-lasso_train_score)*((len(X_train)-1)/(len(X_train)-coeff_used3-1))

In [24]:
pd.DataFrame([[lm1_train_score, lasso_train_score],
              [lm1_train_r2adj, lasso_train_r2adj], 
              [lm1_r2, lasso_test_score],
              [coeff_used1, coeff_used3]],
             columns = ["model 1 - linear regression in Q1","model 3 - lasso regression in Q3"], 
             index = ["R^2 on train set", "Adjusted R^2 on train set", "R^2 on test set", "number of features used"])

,model 1 - linear regression in Q1,model 3 - lasso regression in Q3
R^2 on train set,0.902304,0.858410
Adjusted R^2 on train set,0.897515,0.856597
R^2 on test set,0.890330,0.850822
number of features used,85.000000,23.000000


# Q4

try ridge regression with l2 loss function.

In [25]:
ridge = Ridge()
ridge.fit(X_train,y_train)
ridge1_predictions = ridge.predict(X_test)
ridge_train_score=ridge.score(X_train,y_train)
ridge_test_score=ridge.score(X_test,y_test)
coeff_used4 = np.sum(ridge.coef_!=0)

In [26]:
print("ridge number of features used: ", coeff_used4)
print("ridge training score:", ridge_train_score)
print("ridge test score: ", ridge_test_score)

ridge number of features used:  84
ridge training score: 0.9021865500766303
ridge test score:  0.8907064191902662


In [27]:
ridge_train_r2adj = 1-(1-ridge_train_score)*((len(X_train)-1)/(len(X_train)-coeff_used4-1))
pd.DataFrame([[lm1_train_score, lasso_train_score,ridge_train_score],
              [lm1_train_r2adj, lasso_train_r2adj,ridge_train_r2adj], 
              [lm1_r2, lasso_test_score,ridge_test_score],
              [coeff_used1, coeff_used3,coeff_used4]],
             columns = ["model 1 - linear regression in Q1","model 3 - lasso regression in Q3","model 4 - ridge regression in Q4"], 
             index = ["R^2 on train set", "Adjusted R^2 on train set", "R^2 on test set", "number of features used"])

,model 1 - linear regression in Q1,model 3 - lasso regression in Q3,model 4 - ridge regression in Q4
R^2 on train set,0.902304,0.858410,0.902187
Adjusted R^2 on train set,0.897515,0.856597,0.897451
R^2 on test set,0.890330,0.850822,0.890706
number of features used,85.000000,23.000000,84.000000


# Q5
##### fit a Lasso regression to predict the overall scores of players with an ideal value for alpha. 

In [28]:
lasso = Lasso()

parameters = {'alpha': [1e-15, 1e-10, 1e-8, 1e-4, 1e-3, 3e-3, 4e-3, 5e-3, 7e-3, 1e-2, 0.05, 0.1, 1, 5, 10, 20]}

lasso_regressor = GridSearchCV(lasso, parameters, cv = 5)

lasso_regressor.fit(X_train, y_train);  # lasso_regressor is only a GridSearchCV function, the searching process starts here.

/Users/dengguanying/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3411.6760865910073, tolerance: 7.077029670329669
  positive)
/Users/dengguanying/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3427.8306434580218, tolerance: 7.259854670329671
  positive)
/Users/dengguanying/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3315.408049015944, tolerance: 7.131133997252747
  positive)
/Users/dengguanying/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. 

In [29]:
lasso_regressor.best_params_   #  the best alpha parameter found in GridSearchCV = 0.01

{'alpha': 0.004}

In [30]:
train_score_gscv = lasso_regressor.score(X_train,y_train)  
# the score can only be obtained from a GridSearchCV after running lasso_regressor.fit(... , ...)
# = lasso_regressor.best_estimator_.fit(X_train, y_train).score(X_train, y_train)
test_score_gscv = lasso_regressor.score(X_test,y_test)
coeff_used5 = np.sum(lasso_regressor.best_estimator_.coef_!=0)

train_r2adj_gscv = 1-(1-train_score_gscv)*((len(X_train)-1)/(len(X_train)-coeff_used5-1))
lasso2_predictions = lasso_regressor.predict(X_test)

In [31]:
pd.DataFrame([[lm1_train_score, lasso_train_score, train_score_gscv],
              [lm1_train_r2adj, lasso_train_r2adj, train_r2adj_gscv], 
              [lm1_r2, lasso_test_score, test_score_gscv],
              [coeff_used1, coeff_used3, coeff_used5]],
             columns = ["model 1 - linear regression in Q1","model 3 - lasso regression in Q3", "model 5 - lasso regression with ideal alpha in Q5"], 
             index = ["R^2 on train set", "Adjusted R^2 on train set", "R^2 on test set", "number of features used"])


,model 1 - linear regression in Q1,model 3 - lasso regression in Q3,model 5 - lasso regression with ideal alpha in Q5
R^2 on train set,0.902304,0.858410,0.901005
Adjusted R^2 on train set,0.897515,0.856597,0.897453
R^2 on test set,0.890330,0.850822,0.890345
number of features used,85.000000,23.000000,63.000000


# Q6
##### Calculate AIC and BIC for the models you built in question 1 and question 4.

In [32]:
def AIC(y_true, y_hat, coeff_used):
    resid = y_true - y_hat
    sse = sum(resid**2)
    n = len(y_hat)
    return n*np.log(sse/n) + 2*coeff_used

def BIC(y_true, y_hat, coeff_used):
    resid = y_true - y_hat
    sse = sum(resid**2)
    n = len(y_hat)
    return n*np.log(sse/n) + np.log(n)*coeff_used

In [33]:
#aic and bic of simple linear model in Q1
aic_lm1 = AIC(y_test, lm1_predictions, (coeff_used1+1))
print(aic_lm1)
bic_lm1 = BIC(y_test, lm1_predictions, (coeff_used1+1))
print(bic_lm1)
aicc_lm1 = aic_lm1 + 2*(coeff_used1**2 + coeff_used1)/(len(X_test)-coeff_used1-1)

27251.362641295524
27913.927592318498


In [34]:
#aic and bic of lasso model
aic_lasso2 = AIC(y_test, lasso2_predictions, (coeff_used5+1))
print(aic_lasso2)
bic_lasso2 = BIC(y_test, lasso2_predictions, (coeff_used5+1))
print(bic_lasso2)
aicc_lasso2 = aic_lasso2 + 2*(coeff_used5**2 + coeff_used5)/(len(X_test)-coeff_used5-1)

27205.120053294093
27698.191644753053


In [35]:
pd.DataFrame([[lm1_train_score, train_score_gscv],
              [lm1_train_r2adj, train_r2adj_gscv], 
              [lm1_r2, test_score_gscv],
              [coeff_used1, coeff_used5],
              [aic_lm1, aic_lasso2],
              [bic_lm1, bic_lasso2],
              [aicc_lm1, aicc_lasso2]],
             columns = ["model 1 - linear regression in Q1", "model 5 - lasso regression with ideal alpha in Q5"], 
             index = ["R^2 on train set", "Adjusted R^2 on train set", "R^2 on test set", "number of features used", "AIC","BIC","AICc"])


,model 1 - linear regression in Q1,model 5 - lasso regression with ideal alpha in Q5
R^2 on train set,0.902304,0.901005
Adjusted R^2 on train set,0.897515,0.897453
R^2 on test set,0.890330,0.890345
number of features used,85.000000,63.000000
AIC,27251.362641,27205.120053
BIC,27913.927592,27698.191645
AICc,27252.259519,27205.614080
